![anh1](https://scontent.fhan18-1.fna.fbcdn.net/v/t39.30808-6/527568808_784899461143393_6790818195720877561_n.png?_nc_cat=110&ccb=1-7&_nc_sid=cc71e4&_nc_eui2=AeHP5E0E4DEEVLZ8-gwDOgWYecLcIH_fR7d5wtwgf99Ht2HgvQ0PuYjK_Yi9Tkp7KE8FuUHphBRToDfSxb5URMG5&_nc_ohc=8hkkWWc7tNUQ7kNvwFCAiBN&_nc_oc=Adl0d-l6hr2sOc_3Y1d4a43ZJJ27w73j3w3E_28uI5ubp-gVt5k2PibqVQeIzHtqXgs&_nc_zt=23&_nc_ht=scontent.fhan18-1.fna&_nc_gid=JaJRkErUCv85lJUT14WBRw&oh=00_AfavYKIS4YqSQtjbZVHRHlQ7ln3H7GDiAXp9-LUUOliKww&oe=68BF98AA)

<center>
    <h3>Bài làm vòng 2: Xây dựng một chiến lược lựa chọn cổ phiếu tiềm năng sinh lời</h3>
</center>

<center><h4>Nhóm Votri:</h4></center>
<table border="1">
    <thead>
        <tr>
            <th>Thành viên</th>
            <th>Họ và tên</th>
            <th>Số báo danh</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>1.</td>
            <td>Trương Thị Thùy Dung</td>
            <td>B0426</td>
        </tr>
        <tr>
            <td>2.</td>
            <td>Nguyễn Thế Anh</td>
            <td>B0396</td>
        </tr>
        <tr>
            <td>3.</td>
            <td>Trần Mạnh Hùng</td>
            <td>B0535</td>
        </tr>
    </tbody>
</table>

*Logic cốt lõi của chiến lược là dự báo xác suất thành công của một cơ hội giao dịch dựa trên một kiến trúc Deep Learning tổng hợp, cụ thể 
là CNN + LSTM + Multi-Head Attention với đầu vào là chỉ số kĩ thuật (TA)*

<center>
    <h2>CHUẨN BỊ</h2>
</center>

Nhập, cấu hình các môi trường, thư viện cần thiết:

In [1]:
import os
import gc
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import sys
import math
import numpy as np
import pandas as pd
import datetime
from collections import Counter

# ML libs
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

# optional SOM
try:
    from minisom import MiniSom
    HAS_SOM = True
except Exception:
    HAS_SOM = False

Đăng nhập tài khoản và sử dụng FiinQuantX

In [2]:
from FiinQuantX import FiinSession

username = 'DSTC_36@fiinquant.vn'
password = 'Fiinquant0606'

client = FiinSession(
    username=username,
    password=password,
).login()

fi = client.FiinIndicator()

Cấu hình
- GPU để chạy TensorFlow
- mô hình Deep Learning với Keras

In [3]:
# TF / Keras
import tensorflow as tf
# Configure GPU before any other TF operation
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Enable memory growth for all GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        
        # Set GPU as preferred device
        tf.config.experimental.set_device_policy('explicit')
        print(f"GPU configured successfully. Available GPUs: {len(gpus)}")
        
        # Force specific GPU if multiple available
        with tf.device('/GPU:0'):
            tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
        print("GPU test successful - using GPU for computation")
        
    except RuntimeError as e:
        print(f"GPU configuration error: {e}")
        print("Falling back to CPU")
else:
    print("No GPU detected - using CPU")

# Memory optimization
import psutil
print(f"Available RAM: {psutil.virtual_memory().available / (1024**3):.1f} GB")
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv1D, MaxPooling1D, Dropout, LSTM,
                                     LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D,
                                     Dense, Embedding, Flatten, Concatenate)
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2

No GPU detected - using CPU
Available RAM: 3.1 GB


Sử dụng matplotlib để vẽ đồ thị:

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

Cấu hình và thông số cho pipeline:

In [5]:
CSV_PATH = "DSTC_3y.csv"   # Đường dẫn đến file dữ liệu chứa dữ liệu lịch sử của các cổ phiếu.
START_DATE = None        # Bắt đầu thời gian để lọc data (None thì giữ hết)
END_DATE = None          # Kết thúc thời gian để lọc data
RF_ANNUAL = 0.03          # Tỉ lệ lợi nhuận hàng năm dự tính
MAX_MISSING_RATIO = 0.10          # Tỉ lệ dữ liệu thiếu tối đa cho phép trong các cột dữ liệu, nếu vượt quá sẽ bị loại bỏ
N_RFE_FEATURES = 40          # Số lượng features chọn lựa bằng phương pháp RFE
N_CLUSTERS = 6           # Số clusters khi áp dụng KMeans
FINAL_FEATURE_COUNT = 20           # Số lượng đặc trưng cuối cùng sau khi lựa chọn và lọc các đặc trưng
WINDOW = 40           # Kích thước window cho mô hình deep learning
STEP = 1             # Bước nhảy khi xây dựng các chuỗi dữ liệu
RFE_ESTIMATORS = 100     # Số lượng ước lượng trong RandomForest dùng cho RFE
SOM_ITERS = 200         # Số lần lặp cho Self-Organizing Maps (SOM)
EPOCHS = 100              # Số lượng epochs (vòng lặp) khi huấn luyện mô hình
BATCH_SIZE = 64              # Kích thước của mỗi batch khi huấn luyện mô hình
EMB_DIM = 16              # Kích thước embedding cho mô hình
OUT_DIR = "outputs"                # Thư mục chứa kết quả xuất ra (outputs)
os.makedirs(OUT_DIR, exist_ok=True)
TS = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")               # Chuỗi thời gian (timestamp) duy nhất để đặt tên cho các kết quả

Setting cho phương pháp Triple-Barrier

In [6]:
TAKE_PROFIT_MULT = 2   # Rào cản chốt lời = 3 * ATR (R:R ~ 1:2)
STOP_LOSS_MULT = 1   # Rào cản cắt lỗ = 1 * ATR
MAX_HOLD_PERIOD = 7   # Giữ vị thế tối đa 7 ngày

Setting xử lý data

In [7]:
CHUNK_SIZE = 10000       # Số lượng samples tối đa trong mỗi phần dữ liệu khi tải vào bộ nhớ
MAX_SAMPLES = 50000      # Giới hạn số lượng samples sẽ được sử dụng trong training để tránh quá tải bộ nhớ OOM

## Khai báo các hàm cần thiết cho chương trình

Hàm `check_memory()` theo dõi việc sử dụng bộ nhớ và đảm bảo không quá tải bộ nhớ trong quá trình xử lý dữ liệu:

In [8]:
def check_memory():
    """Monitor memory usage"""
    mem = psutil.virtual_memory()  # Lấy thông tin bộ nhớ hệ thống
    print(f"Memory usage: {mem.percent:.1f}% ({(mem.total - mem.available)/(1024**3):.1f}/{mem.total/(1024**3):.1f} GB)")  
    # In ra tỉ lệ sử dụng bộ nhớ và tổng bộ nhớ, bộ nhớ còn lại
    if mem.percent > 85:  # Nếu sử dụng bộ nhớ trên 85%
        print("WARNING: High memory usage detected!")  # Cảnh báo sử dụng bộ nhớ cao
        gc.collect()  # Dọn dẹp bộ nhớ (garbage collection)
        return True  # Trả về True khi bộ nhớ cao
    return False  # Trả về False nếu bộ nhớ dưới mức cảnh báo

Hàm `load_data()` đọc dữ liệu từ tệp CSV, phân mảnh để quản lý bộ nhớ

In [9]:
def load_data(path):
    if not os.path.exists(path):  # Kiểm tra xem file có tồn tại không
        raise FileNotFoundError(path)  # Nếu không có file, raise lỗi

    # Load in chunks to manage memory
    print("Loading data in chunks...")  # In ra thông báo đang tải dữ liệu theo từng phần
    chunk_list = []  # Danh sách lưu các phần dữ liệu (chunks)
    
    try:
        for chunk in pd.read_csv(path, chunksize=CHUNK_SIZE):  # Đọc dữ liệu từ file theo từng phần
            if 'timestamp' in chunk.columns:  # Kiểm tra xem cột 'timestamp' có trong dữ liệu không
                chunk['timestamp'] = pd.to_datetime(chunk['timestamp'])  # Chuyển đổi cột 'timestamp' sang kiểu datetime
            chunk_list.append(chunk)  # Thêm phần dữ liệu vào danh sách
            
            # Kiểm tra bộ nhớ sau khi đọc mỗi phần
            if check_memory():  # Nếu bộ nhớ cao, dừng việc đọc thêm dữ liệu
                print("Memory pressure detected, reducing chunk size...")
                break  # Dừng vòng lặp nếu bộ nhớ quá tải
                
        df = pd.concat(chunk_list, ignore_index=True)  # Kết hợp tất cả các phần dữ liệu thành một DataFrame
        del chunk_list  # Xóa chunk_list để giải phóng bộ nhớ
        gc.collect()  # Dọn dẹp bộ nhớ sau khi kết hợp dữ liệu
        
        # Giới hạn số lượng mẫu nếu quá nhiều
        if len(df) > MAX_SAMPLES:  # Nếu số lượng mẫu vượt quá giới hạn MAX_SAMPLES
            print(f"Limiting dataset from {len(df)} to {MAX_SAMPLES} samples")  # Thông báo sẽ giới hạn số mẫu
            df = df.sample(n=MAX_SAMPLES, random_state=42).sort_values(['ticker', 'timestamp']).reset_index(drop=True)
            # Giới hạn số mẫu và sắp xếp lại theo 'ticker' và 'timestamp'
            
        print(f"Data loaded: {df.shape}")  # In ra kích thước dữ liệu đã tải
        return df  # Trả về DataFrame chứa dữ liệu đã tải
        
    except Exception as e:  # Nếu có lỗi khi đọc dữ liệu
        print(f"Error loading data: {e}")  # In ra lỗi
        # Thử tải một phần dữ liệu nhỏ hơn
        print("Attempting to load smaller subset...")
        df = pd.read_csv(path, nrows=MAX_SAMPLES//2)  # Đọc một nửa số mẫu nếu có lỗi
        if 'timestamp' in df.columns:  # Kiểm tra lại và chuyển đổi cột 'timestamp'
            df['timestamp'] = pd.to_datetime(df['timestamp'])
        return df  # Trả về DataFrame đã tải một phần

Hàm `drop_high_missing()` loại bỏ các cột trong DataFrame có tỉ lệ missing value vượt quá mức quy định (max_ratio)

In [10]:
def drop_high_missing(df, max_ratio):
    missing = df.isna().mean()  # Tính toán tỉ lệ missing value (giá trị thiếu) trên mỗi cột
    drop_cols = missing[missing > max_ratio].index.tolist()  # Chọn các cột có tỉ lệ missing value vượt quá max_ratio
    if drop_cols:  # Nếu có cột nào có missing > max_ratio, tiến hành loại bỏ
        print(f"Dropping {len(drop_cols)} columns with >{max_ratio*100:.1f}% missing")
        df = df.drop(columns=drop_cols)  # Loại bỏ các cột có missing value cao
    return df  # Trả về DataFrame đã loại bỏ các cột không cần thiết

Hàm `interp_per_ticker()` thay thế các giá trị thiếu trong các cột số (numeric_cols) bằng phương pháp nội suy (interpolation), và nội suy được thực hiện theo từng nhóm ticker

In [11]:
def interp_per_ticker(df, numeric_cols):
    # interpolate per ticker to avoid leaking across tickers
    for t, g in df.groupby('ticker'):  # Nhóm dữ liệu theo từng 'ticker'
        idx = g.index  # Lấy chỉ số của nhóm
        df.loc[idx, numeric_cols] = g[numeric_cols].interpolate(method='linear', limit_direction='both').ffill().bfill().values
        # Sử dụng phương pháp nội suy (interpolation) tuyến tính để thay thế giá trị thiếu trong các cột numeric_cols
    return df  # Trả về DataFrame đã được nội suy

#### Về Triple-Barrier Method

Phương pháp Triple-Barrier bao gồm ba yếu tố quan trọng:
- Take Profit (TP): Mức giá khi giá đạt đến sẽ chốt lời
  
- Stop Loss (SL): Mức giá khi giá đạt đến sẽ cắt lỗ

  
- Timeout: Thời gian tối đa được phép giữ vị thế (trade), nếu không đạt TP hoặc SL trong khoảng thời gian này, thì coi như hết thời gian

Phương pháp Triple-Barrier để tạo nhãn (labels) cho mỗi cổ phiếu trong dữ liệu, dùng để phân loại các tín hiệu mua bán. Mục đích chính là xác định khi nào chốt lời (take profit), cắt lỗ (stop loss), hoặc hết thời gian giữ (timeout)

Hàm `apply_triple_barrier()` thực hiện phương pháp **Triple-Barrier** để tạo nhãn cho mỗi cổ phiếu trong dữ liệu. Mỗi nhãn có thể là:
+ 1 (win): Nếu mức chốt lời (take profit) được đạt
  
+ 0 (loss): Nếu mức cắt lỗ (stop loss) được đạt

+ 2 (timeout): Nếu hết thời gian giữ vị thế mà không đạt cả TP và SL



In [12]:
def apply_triple_barrier(df, tp_mult=1.5, sl_mult=1.5, max_period=10):
    """
    Applies the Triple-Barrier Method to generate labels for each ticker.

    Args:
        df (pd.DataFrame): DataFrame containing price data and 'atr_14'.
        tp_mult (float): Multiplier for ATR to set the take-profit barrier.
        sl_mult (float): Multiplier for ATR to set the stop-loss barrier.
        max_period (int): Maximum number of days to hold the position.

    Returns:
        pd.Series: A series with labels (1 for win, 0 for loss, 2 for timeout).
    """
    print(f"Applying Triple-Barrier with TP={tp_mult}*ATR, SL={sl_mult}*ATR, Max Hold={max_period} days...")
    
    # Create an empty Series to store the results, ensuring it can handle NaNs before filling
    all_labels = pd.Series(np.nan, index=df.index)

    # Process each ticker individually
    for ticker, group in df.groupby('ticker'):
        group = group.sort_index() # Ensure data is sorted by time
        labels = pd.Series(np.nan, index=group.index)
        
        # Iterate through each time point to start a "trade"
        for i in range(len(group) - 1):
            entry_idx = group.index[i]
            entry_price = group.loc[entry_idx, 'close']
            atr = group.loc[entry_idx, 'atr_14']

            # Skip if ATR is missing or zero
            if pd.isna(atr) or atr == 0:
                continue

            # 1. Define the barriers
            upper_barrier = entry_price + (atr * tp_mult)
            lower_barrier = entry_price - (atr * sl_mult)

            # 2. Define the time window
            window_end_idx = min(i + max_period, len(group) - 1)
            window_indices = group.index[i+1 : window_end_idx+1]
            price_window = group.loc[window_indices]

            # 3. Find the time when barriers are hit
            hit_tp = price_window[price_window['high'] >= upper_barrier]
            hit_sl = price_window[price_window['low'] <= lower_barrier]

            time_to_tp = hit_tp.index.min() if not hit_tp.empty else pd.NaT
            time_to_sl = hit_sl.index.min() if not hit_sl.empty else pd.NaT
            
            # 4. Determine the label
            # Case 1: Both barriers are hit
            if pd.notna(time_to_tp) and pd.notna(time_to_sl):
                # Choose the one that was hit first
                if time_to_tp < time_to_sl:
                    labels.loc[entry_idx] = 1 # Win
                else:
                    labels.loc[entry_idx] = 0 # Loss
            # Case 2: Only Take Profit is hit
            elif pd.notna(time_to_tp):
                labels.loc[entry_idx] = 1 # Win
            # Case 3: Only Stop Loss is hit
            elif pd.notna(time_to_sl):
                labels.loc[entry_idx] = 0 # Loss
            # Case 4: No barrier is hit (Timeout)
            else:
                labels.loc[entry_idx] = 2 # Timeout

        all_labels.update(labels)
    
    return all_labels

Hàm `time_split_train_mask()` chia dữ liệu theo thời gian dựa trên cột `timestamp`, và tạo ra một mask (chuỗi boolean) để xác định những mẫu nào sẽ được dùng cho huấn luyện và kiểm tra. Tỉ lệ dữ liệu huấn luyện được xác định bởi tham số `train_frac`, và dữ liệu được chia theo thời gian, không gian (random).

In [13]:
def time_split_train_mask(df, train_frac=0.70):
    # compute global train end date by unique sorted dates
    if 'timestamp' not in df.columns:
        raise RuntimeError('timestamp column required for time split')
    unique_days = np.sort(df['timestamp'].dt.floor('D').unique())
    if len(unique_days) < 3:
        raise RuntimeError('Too few unique days to split')
    train_end = unique_days[int(len(unique_days)*train_frac)-1]
    mask = df['timestamp'].dt.floor('D') <= train_end
    print(f"Time-split train_end date: {train_end} -> train rows: {mask.sum()} / {len(df)}")
    return mask

Hàm `train_only_rfe()` thực hiện việc chọn lọc các đặc trưng quan trọng từ dữ liệu bằng phương pháp **RFE** với mô hình **RandomForestClassifier**. Nó giúp loại bỏ các đặc trưng không cần thiết và chỉ giữ lại những đặc trưng quan trọng, có ảnh hưởng lớn đến dự đoán. Kết quả là một danh sách các đặc trưng quan trọng cùng với mô hình **RFE** đã huấn luyện.

In [14]:
def train_only_rfe(df, numeric_cols, train_mask, n_select=64, n_estimators=100):
    X = df.loc[train_mask, numeric_cols].fillna(0.0).values
    y = df.loc[train_mask, 'target'].values
    rf = RandomForestClassifier(n_estimators=n_estimators, random_state=42, n_jobs=-1)
    n_select = min(n_select, X.shape[1])
    rfe = RFE(estimator=rf, n_features_to_select=n_select, step=1)
    print("Fitting RFE on train rows... (this may take time)")
    rfe.fit(X, y)
    feature_ranking = sorted(zip(rfe.ranking_, numeric_cols))
    print("\nFeature Ranking by RFE:")
    for rank, name in feature_ranking:
        print(f"Rank {rank}: {name}")
    keep = [c for c, s in zip(numeric_cols, rfe.support_) if s]
    return keep, rfe

Hàm `kmeans_cluster_features()` sử dụng thuật toán **KMeans** để phân nhóm các đặc trưng của dữ liệu huấn luyện thành các cụm (clusters). Việc phân nhóm này có thể giúp trong việc lựa chọn các đặc trưng quan trọng hoặc giảm chiều dữ liệu. Hàm trả về nhãn phân nhóm của các đặc trưng, đối tượng `scaler` để chuẩn hóa dữ liệu và đối tượng k để tra cứu thêm thông tin về mô hình **KMeans**.

In [15]:
def kmeans_cluster_features(X_train_selected, n_clusters=5):
    # X_train_selected: DataFrame of selected features (train rows only)
    # Chuẩn hóa dữ liệu:
    scaler = StandardScaler()
    Xs = scaler.fit_transform(X_train_selected.values)
    # cluster across features -> transpose
    k = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    labels = k.fit_predict(Xs.T)
    return labels, scaler, k

Hàm `som_select_per_cluster()` sử dụng **Self-Organizing Map (SOM)** (nếu có sẵn thư viện `MiniSom`) để chọn lọc các đặc trưng quan trọng từ mỗi nhóm (cluster). Nếu không sử dụng SOM, nó sẽ chọn các đặc trưng dựa trên độ tương quan giữa các đặc trưng. Cuối cùng, hàm trả về danh sách các đặc trưng được chọn từ mỗi nhóm, giúp giảm số lượng đặc trưng mà vẫn đảm bảo tính hiệu quả trong mô hình.

In [16]:
def som_select_per_cluster(X_train_selected, labels_cluster, feature_names, target_count=10):
    # X_train_selected: DataFrame (train_rows x features)
    chosen = []
    features = np.array(feature_names)  # Chuyển tên feature thành mảng numpy
    for cl in np.unique(labels_cluster):  # Lặp qua từng nhóm phân loại
        idxs = np.where(labels_cluster == cl)[0]  # Tìm các chỉ số (index) thuộc nhóm hiện tại
        if len(idxs) == 0:  # Nếu không có feature nào thuộc nhóm, bỏ qua
            continue
        feats = features[idxs]  # Lấy các feature thuộc nhóm
        cluster_data = X_train_selected[feats].values  # Dữ liệu của các feature trong nhóm
        # transpose -> (n_feats, n_train_rows)
        v = cluster_data.T  # Chuyển vị dữ liệu: đặc trưng -> mẫu
        eps = 1e-8  # Hệ số nhỏ để tránh chia cho 0
        v = (v - v.mean(axis=1, keepdims=True)) / (v.std(axis=1, keepdims=True) + eps)  # Chuẩn hóa từng feature
        n_feats = v.shape[0]
        som_x = max(1, int(math.sqrt(n_feats)))  # Tính toán kích thước lưới SOM
        som_y = som_x
        if HAS_SOM:  # Kiểm tra xem có thư viện MiniSom hay không
            som = MiniSom(som_x, som_y, v.shape[1], sigma=0.5, learning_rate=0.5)  # Khởi tạo SOM
            som.random_weights_init(v)  # Khởi tạo trọng số ngẫu nhiên
            som.train_random(v, SOM_ITERS)  # Huấn luyện SOM
            mapped = [som.winner(v_i) for v_i in v]  # Tìm vị trí thắng (winner) của mỗi đặc trưng
            node_counts = Counter(mapped)  # Đếm số lượng đặc trưng được ánh xạ tới mỗi node
            nodes_sorted = [n for n, _ in node_counts.most_common()]  # Sắp xếp các node theo số lượng đặc trưng được ánh xạ tới
            picks = []  # Danh sách đặc trưng được chọn
            for node in nodes_sorted:
                idxs_node = [i for i, m in enumerate(mapped) if m == node]  # Các đặc trưng ánh xạ đến node hiện tại
                for ii in idxs_node:
                    f = feats[ii]
                    if f not in picks:
                        picks.append(f)  # Thêm đặc trưng vào danh sách nếu chưa có
                    if len(picks) >= max(1, int(len(feats) * target_count / max(1, len(feature_names)))):  # Đảm bảo không vượt quá target_count
                        break
                if len(picks) >= max(1, int(len(feats) * target_count / max(1, len(feature_names)))):  # Dừng nếu đã đủ số lượng đặc trưng
                    break
            if len(picks) == 0:
                # fallback choose top variance
                var_idx = np.argsort(-np.var(v, axis=1))[:1]
                picks = [feats[i] for i in var_idx]
        else:
            # fallback: chọn các đặc trưng có tương quan cao nhất
            if v.shape[0] == 1:
                picks = [feats[0]]
            else:
                corr = np.corrcoef(v)  # Tính ma trận tương quan
                avg_abs = np.nanmean(np.abs(corr), axis=1)  # Tính trung bình giá trị tuyệt đối của tương quan
                order = np.argsort(-avg_abs)  # Sắp xếp các đặc trưng theo tương quan
                n_pick = max(1, int(len(feats) * target_count / max(1, len(feature_names))))  # Đảm bảo số lượng đặc trưng không vượt quá target_count
                picks = [feats[i] for i in order[:n_pick]]  # Chọn các đặc trưng có tương quan cao nhất
        chosen.extend(picks)
    # dedupe preserving order
    seen = set()  # Set dùng để kiểm tra các đặc trưng đã chọn
    final = [x for x in chosen if not (x in seen or seen.add(x))]  # Loại bỏ các đặc trưng trùng lặp
    return final[:target_count]

Hàm `correlation_filter()` kiểm tra tính tương quan giữa các đặc trưng trong `df_full`. Nếu hai đặc trưng có hệ số tương quan tuyệt đối vượt quá ngưỡng `threshold`, một trong hai đặc trưng sẽ bị loại bỏ. Kết quả là một danh sách các đặc trưng không có sự dư thừa về thông tin, giúp cải thiện hiệu suất của mô hình học máy và tránh hiện tượng **multicollinearity**.

In [17]:
def correlation_filter(df_full, features, threshold=0.8):
    sub = df_full[features].corr().abs()
    to_drop = set()
    for i in range(len(features)):
        for j in range(i+1, len(features)):
            if sub.iloc[i,j] > threshold:
                # drop j (later) to keep earlier
                to_drop.add(features[j])
    filtered = [f for f in features if f not in to_drop]
    return filtered

#### Hàm tính toán các chỉ số kỹ thuật (TA) và cơ bản (FA)

Hàm `compute_indicators()` tính toán và thêm vào DataFrame các chỉ số kỹ thuật như *Moving Averages*, *RSI*, *ATR*, *MACD*, *Bollinger Bands*, *ADX*, *Ichimoku*, và các chỉ số khối lượng như *OBV* và *MFI*. Nó sử dụng thư viện **FiinQuantX** có sẵn và cả tự tính toán bằng **pandas**. Những chỉ số này rất quan trọng trong phân tích kỹ thuật và giúp xây dựng các mô hình học máy trong dự đoán giá cổ phiếu hoặc tài sản.

In [18]:
def compute_indicators(df):
    """Compute a compact set of technical indicators used by the pipeline.
    Adds columns in-place to df. Implementations are lightweight and avoid external APIs.
    """
    # ensure timestamp
    if 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'])

    # If a FiinQuantX indicator object exists in globals, prefer it
    use_fi = 'fi' in globals() and globals().get('fi') is not None

    # 1) Moving averages + WMA
    ma_windows = [5, 10, 20, 50]
    for w in ma_windows:
        if use_fi:
            try:
                df[f'ema_{w}'] = fi.ema(df['close'], window=w)
                df[f'sma_{w}'] = fi.sma(df['close'], window=w)
                df[f'wma_{w}'] = fi.wma(df['close'], window=w)
                continue
            except Exception:
                pass
        # pandas group-based EMAs/SMA
        df[f'ema_{w}'] = df.groupby('ticker')['close'].transform(lambda x, p=w: x.ewm(span=p, adjust=False).mean())
        df[f'sma_{w}'] = df.groupby('ticker')['close'].transform(lambda x, p=w: x.rolling(window=p, min_periods=1).mean())
        # WMA implementation
        def _wma(s, p=w):
            weights = np.arange(1, p+1)
            return s.rolling(p, min_periods=1).apply(lambda x: np.dot(x, weights[-len(x):]) / weights[-len(x):].sum(), raw=True)
        df[f'wma_{w}'] = df.groupby('ticker')['close'].transform(lambda x, p=w: _wma(x, p))

    # RSI
    def _rsi(series, period=14):
        delta = series.diff()
        up = delta.clip(lower=0)
        down = -delta.clip(upper=0)
        ma_up = up.ewm(alpha=1/period, adjust=False).mean()
        ma_down = down.ewm(alpha=1/period, adjust=False).mean()
        rs = ma_up / (ma_down + 1e-8)
        return 100 - (100 / (1 + rs))

    for w in [7, 14, 30]:
        if use_fi:
            try:
                df[f'rsi_{w}'] = fi.rsi(df['close'], window=w)
                continue
            except Exception:
                pass
        df[f'rsi_{w}'] = df.groupby('ticker')['close'].transform(lambda x, p=w: _rsi(x, p))

    # ATR
    def _atr(g, period=14):
        high = g['high']; low = g['low']; close = g['close']
        prev_close = close.shift(1)
        tr = pd.concat([high - low, (high - prev_close).abs(), (low - prev_close).abs()], axis=1).max(axis=1)
        return tr.rolling(window=period, min_periods=1).mean()

    for w in [7, 14, 21]:
        if use_fi:
            try:
                df[f'atr_{w}'] = fi.atr(df['high'], df['low'], df['close'], window=w)
                continue
            except Exception:
                pass
        df[f'atr_{w}'] = df.groupby('ticker').apply(lambda g, p=w: _atr(g, period=p)).reset_index(level=0, drop=True)

    # Bollinger Bands
    bb_windows = [10, 20, 30]
    bb_devs = [1.5, 2.0, 2.5]
    for w in bb_windows:
        for d in bb_devs:
            if use_fi:
                try:
                    df[f'bollinger_hband_{w}_{d}'] = fi.bollinger_hband(df['close'], window=w, window_dev=d)
                    df[f'bollinger_lband_{w}_{d}'] = fi.bollinger_lband(df['close'], window=w, window_dev=d)
                    continue
                except Exception:
                    pass
            m = df.groupby('ticker')['close'].transform(lambda x, p=w: x.rolling(window=p, min_periods=1).mean())
            s = df.groupby('ticker')['close'].transform(lambda x, p=w: x.rolling(window=p, min_periods=1).std())
            df[f'bollinger_hband_{w}_{d}'] = m + d * s
            df[f'bollinger_lband_{w}_{d}'] = m - d * s

    # MACD variants
    def _ema(series, span):
        return series.ewm(span=span, adjust=False).mean()

    macd_params = [(12, 26, 9), (5, 35, 5), (20, 50, 10)]
    for fast, slow, signal in macd_params:
        name = f"{fast}_{slow}_{signal}"
        if use_fi:
            try:
                df[f'macd_{name}'] = fi.macd(df['close'], window_fast=fast, window_slow=slow)
                df[f'macd_signal_{name}'] = fi.macd_signal(df['close'], window_fast=fast, window_slow=slow, window_sign=signal)
                df[f'macd_diff_{name}'] = fi.macd_diff(df['close'], window_fast=fast, window_slow=slow, window_sign=signal)
                continue
            except Exception:
                pass
        df[f'macd_{name}'] = df.groupby('ticker')['close'].transform(lambda x, f=fast, s=slow: _ema(x, f) - _ema(x, s))
        df[f'macd_signal_{name}'] = df.groupby('ticker')['macd_' + name].transform(lambda x, p=signal: _ema(x, p))
        df[f'macd_diff_{name}'] = df[f'macd_{name}'] - df[f'macd_signal_{name}']

    # Stochastic Oscillator and signal
    for w in [10, 14, 20]:
        if use_fi:
            try:
                df[f'stoch_{w}'] = fi.stoch(df['high'], df['low'], df['close'], window=w)
                df[f'stoch_signal_{w}'] = fi.stoch_signal(df['high'], df['low'], df['close'], window=w, smooth_window=3)
                continue
            except Exception:
                pass
        k = df.groupby('ticker').apply(lambda g, p=w: (g['close'] - g['low'].rolling(p, min_periods=1).min()) / (g['high'].rolling(p, min_periods=1).max() - g['low'].rolling(p, min_periods=1).min() + 1e-8)).reset_index(level=0, drop=True)
        df[f'stoch_{w}'] = k
        df[f'stoch_signal_{w}'] = df.groupby('ticker')[f'stoch_{w}'].transform(lambda x: x.rolling(3, min_periods=1).mean())

    # ADX and DI
    def _compute_adx(g, period=14):
        high = g['high']; low = g['low']; close = g['close']
        prev_high = high.shift(1); prev_low = low.shift(1); prev_close = close.shift(1)
        plus_dm = (high - prev_high).where((high - prev_high) > (prev_low - low), 0.0)
        minus_dm = (prev_low - low).where((prev_low - low) > (high - prev_high), 0.0)
        tr = pd.concat([high - low, (high - prev_close).abs(), (low - prev_close).abs()], axis=1).max(axis=1)
        atr = tr.rolling(window=period, min_periods=1).mean()
        plus_di = 100 * (plus_dm.rolling(window=period, min_periods=1).mean() / (atr + 1e-8))
        minus_di = 100 * (minus_dm.rolling(window=period, min_periods=1).mean() / (atr + 1e-8))
        dx = 100 * (plus_di - minus_di).abs() / (plus_di + minus_di + 1e-8)
        adx = dx.rolling(window=period, min_periods=1).mean()
        return adx, plus_di, minus_di

    for w in [10, 14, 20]:
        if use_fi:
            try:
                df[f'adx_{w}'] = fi.adx(df['high'], df['low'], df['close'], window=w)
                df[f'adx_pos_{w}'] = fi.adx_pos(df['high'], df['low'], df['close'], window=w)
                df[f'adx_neg_{w}'] = fi.adx_neg(df['high'], df['low'], df['close'], window=w)
                continue
            except Exception:
                pass
        tmp = df.groupby('ticker').apply(lambda g, p=w: _compute_adx(g, period=p)).reset_index(level=0, drop=True)
        # tmp is tuple series; unpack
        df[f'adx_{w}'] = tmp.apply(lambda x: x[0])
        df[f'adx_pos_{w}'] = tmp.apply(lambda x: x[1])
        df[f'adx_neg_{w}'] = tmp.apply(lambda x: x[2])

    # Ichimoku (basic components)
    def _ichimoku(g, p1=9, p2=26, p3=52):
        conv = (g['high'].rolling(p1, min_periods=1).max() + g['low'].rolling(p1, min_periods=1).min()) / 2
        base = (g['high'].rolling(p2, min_periods=1).max() + g['low'].rolling(p2, min_periods=1).min()) / 2
        span_a = ((conv + base) / 2).shift(p2)
        span_b = ((g['high'].rolling(p3, min_periods=1).max() + g['low'].rolling(p3, min_periods=1).min()) / 2).shift(p2)
        return conv, base, span_a, span_b

    def _ichimoku(g, p1=9, p2=26, p3=52):
        conv = (g['high'].rolling(p1, min_periods=1).max() + g['low'].rolling(p1, min_periods=1).min()) / 2
        base = (g['high'].rolling(p2, min_periods=1).max() + g['low'].rolling(p2, min_periods=1).min()) / 2
        span_a = ((conv + base) / 2).shift(p2)
        span_b = ((g['high'].rolling(p3, min_periods=1).max() + g['low'].rolling(p3, min_periods=1).min()) / 2).shift(p2)
        return conv, base, span_a, span_b

    # Prefer vendor functions; if they fail, compute per-group and assign to preserve indices
    if use_fi:
        try:
            df['senkou_span_a'] = fi.ichimoku_a(df['high'], df['low'], df['close'], window1=9, window2=26, window3=52)
            df['senkou_span_b'] = fi.ichimoku_b(df['high'], df['low'], df['close'], window1=9, window2=26, window3=52)
            df['kijun_sen'] = fi.ichimoku_base_line(df['high'], df['low'], df['close'], window1=9, window2=26, window3=52)
            df['tenkan_sen'] = fi.ichimoku_conversion_line(df['high'], df['low'], df['close'], window1=9, window2=26, window3=52)
        except Exception:
            use_fi = False

    if not use_fi:
        # compute per-ticker and assign by index to avoid misalignment
        df['tenkan_sen'] = np.nan
        df['kijun_sen'] = np.nan
        df['senkou_span_a'] = np.nan
        df['senkou_span_b'] = np.nan
        for t, g in df.groupby('ticker'):
            conv, base, span_a, span_b = _ichimoku(g)
            idx = g.index
            df.loc[idx, 'tenkan_sen'] = conv.values
            df.loc[idx, 'kijun_sen'] = base.values
            df.loc[idx, 'senkou_span_a'] = span_a.values
            df.loc[idx, 'senkou_span_b'] = span_b.values
    # Volume indicators
    if use_fi:
        try:
            df['obv'] = fi.obv(df['close'], df['volume'])
        except Exception:
            df['obv'] = df.groupby('ticker').apply(lambda g: (np.sign(g['close'].diff()) * g['volume']).fillna(0).cumsum()).reset_index(level=0, drop=True)
        try:
            df['mfi_14'] = fi.mfi(df['high'], df['low'], df['close'], df['volume'], window=14)
        except Exception:
            # fallback simple MFI approximation
            def _mfi(g, period=14):
                typical = (g['high'] + g['low'] + g['close']) / 3.0
                money = typical * g['volume']
                pos = (typical > typical.shift(1)).astype(int)
                mf_pos = money.where(pos==1, 0).rolling(window=period, min_periods=1).sum()
                mf_neg = money.where(pos==0, 0).rolling(window=period, min_periods=1).sum()
                mfr = mf_pos / (mf_neg + 1e-8)
                return 100 - (100 / (1 + mfr))
            df['mfi_14'] = df.groupby('ticker').apply(lambda g: _mfi(g)).reset_index(level=0, drop=True)
        try:
            df['vwap_14'] = fi.vwap(df['high'], df['low'], df['close'], df['volume'], window=14)
        except Exception:
            df['vwap_14'] = df.groupby('ticker').apply(lambda g: (g['close'] * g['volume']).rolling(window=14, min_periods=1).sum() / (g['volume'].rolling(window=14, min_periods=1).sum() + 1e-8)).reset_index(level=0, drop=True)
    else:
        df['obv'] = df.groupby('ticker').apply(lambda g: (np.sign(g['close'].diff()) * g['volume']).fillna(0).cumsum()).reset_index(level=0, drop=True)
        def _mfi(g, period=14):
            typical = (g['high'] + g['low'] + g['close']) / 3.0
            money = typical * g['volume']
            pos = (typical > typical.shift(1)).astype(int)
            mf_pos = money.where(pos==1, 0).rolling(window=period, min_periods=1).sum()
            mf_neg = money.where(pos==0, 0).rolling(window=period, min_periods=1).sum()
            mfr = mf_pos / (mf_neg + 1e-8)
            return 100 - (100 / (1 + mfr))
        df['mfi_14'] = df.groupby('ticker').apply(lambda g: _mfi(g)).reset_index(level=0, drop=True)
        df['vwap_14'] = df.groupby('ticker').apply(lambda g: (g['close'] * g['volume']).rolling(window=14, min_periods=1).sum() / (g['volume'].rolling(window=14, min_periods=1).sum() + 1e-8)).reset_index(level=0, drop=True)

    # Advanced / vendor-specific indicators: prefer fi.* implementations when available
    adv_names = ['supertrend_14', 'zigzag', 'fvg', 'liquidity']
    for name in adv_names:
        if use_fi:
            try:
                if name == 'supertrend_14':
                    df[name] = fi.supertrend(df['high'], df['low'], df['close'], window=14)
                elif name == 'zigzag':
                    df[name] = fi.zigzag(df['high'], df['low'], dev_threshold=5.0, depth=10)
                elif name == 'fvg':
                    df[name] = fi.fvg(df['open'], df['high'], df['low'], df['close'], join_consecutive=True)
                elif name == 'liquidity':
                    df[name] = fi.liquidity(df['open'], df['high'], df['low'], df['close'])
                continue
            except Exception:
                pass
        # Fallbacks: simple placeholders or NaN
        if name == 'supertrend_14':
            # simple supertrend-like: close - rolling median of ATR
            df[name] = df[f'atr_14'] * 0.0
        else:
            df[name] = np.nan

    # Ensure volume exists
    if 'volume' not in df.columns:
        df['volume'] = 0.0

    # per-ticker stats used for embedding/score
    if 'ticker' in df.columns:
        df['ret'] = df.groupby('ticker')['close'].pct_change()
        ticker_stats = df.groupby('ticker').agg(
            mean_ret=('ret', 'mean'),
            vol_ret=('ret', 'std'),
            pct_pos=('ret', lambda x: (x > 0).mean()),
            avg_vol=('volume', 'mean'),
            count=('ret', 'count')
        ).reset_index()
        ticker_stats['sharpe_like'] = ticker_stats['mean_ret'] / ticker_stats['vol_ret'].replace(0, np.nan).fillna(1e-6)
        _sc = MinMaxScaler()
        ticker_stats['ticker_score'] = _sc.fit_transform(ticker_stats[['sharpe_like']].fillna(0))
        df = df.merge(ticker_stats[['ticker', 'ticker_score']], on='ticker', how='left')
        ticker_to_id = {t: i for i, t in enumerate(sorted(df['ticker'].unique()))}
        df['ticker_id'] = df['ticker'].map(ticker_to_id)

    # --- New Feature Engineering Section ---
    print("Generating advanced features...")
    
    # a. Stationarity Features (Rolling Z-Scores)
    z_score_window = 30
    indicators_to_zscore = ['rsi_14', f'macd_{macd_params[0][0]}_{macd_params[0][1]}_{macd_params[0][2]}', 'stoch_14', 'mfi_14']
    
    # FIX: Ensure columns are numeric before performing rolling calculations
    for indicator in indicators_to_zscore:
        if indicator in df.columns:
            df[indicator] = pd.to_numeric(df[indicator], errors='coerce')

    for indicator in indicators_to_zscore:
        if indicator in df.columns:
            rolling_mean = df.groupby('ticker')[indicator].transform(lambda x: x.rolling(window=z_score_window).mean())
            rolling_std = df.groupby('ticker')[indicator].transform(lambda x: x.rolling(window=z_score_window).std())
            df[f'{indicator}_zscore'] = (df[indicator] - rolling_mean) / (rolling_std + 1e-8)

    # b. Relational Features
    if 'ema_20' in df.columns and 'ema_5' in df.columns:
        df['price_vs_ema20'] = df['close'] / df['ema_20']
        df['ema5_vs_ema20'] = df['ema_5'] / df['ema_20']
    
    if 'bollinger_lband_20_2.0' in df.columns and 'bollinger_hband_20_2.0' in df.columns:
        df['price_in_bb'] = (df['close'] - df['bollinger_lband_20_2.0']) / (df['bollinger_hband_20_2.0'] - df['bollinger_lband_20_2.0'] + 1e-8)

    # c. Market Regime Features
    if 'atr_14' in df.columns:
        df['normalized_atr_14'] = df['atr_14'] / df['close']
    
    if 'ret' in df.columns:
        df['volatility_60d'] = df.groupby('ticker')['ret'].transform(lambda x: x.rolling(60).std())
        
    if 'adx_14' in df.columns:
        df['is_trending'] = (df['adx_14'] > 25).astype(int)
        
    print("Advanced features generated.")
    # --- End of New Feature Engineering Section ---

    # --- Start of Super Advanced Feature Engineering ---
    print("Generating interaction, confirmation, and RoC features...")

    # a. Interaction Features
    if 'rsi_14' in df.columns and 'adx_14' in df.columns:
        df['rsi_x_adx14'] = df['rsi_14'] * df['adx_14']

    if 'volume' in df.columns and 'volatility_60d' in df.columns:
        # Normalize volume before multiplying
        df['volume_norm'] = df.groupby('ticker')['volume'].transform(lambda x: (x - x.mean()) / (x.std() + 1e-8))
        df['vol_x_volatility'] = df['volume_norm'] * df['volatility_60d']

    # b. Confirmation Features
    # Ensure all necessary columns exist before creating the list
    bullish_conditions = []
    if 'ema_50' in df.columns:
        bullish_conditions.append(df['close'] > df['ema_50'])
    if f'macd_diff_12_26_9' in df.columns:
        bullish_conditions.append(df[f'macd_diff_12_26_9'] > 0)
    if 'rsi_14' in df.columns:
        bullish_conditions.append(df['rsi_14'] > 50)
    if 'adx_pos_14' in df.columns and 'adx_neg_14' in df.columns:
        bullish_conditions.append(df['adx_pos_14'] > df['adx_neg_14'])
    
    if bullish_conditions:
        df['bullish_confirmation_score'] = np.sum(bullish_conditions, axis=0)

    # c. Rate of Change (RoC) Features
    if 'ema_5' in df.columns:
        df['ema_5_roc_10'] = df.groupby('ticker')['ema_5'].transform(lambda x: x.pct_change(10))
    if 'rsi_14' in df.columns:
        df['rsi_14_roc_5'] = df.groupby('ticker')['rsi_14'].transform(lambda x: x.pct_change(5))
        
    print("Super advanced features generated.")
    # --- End of Super Advanced Feature Engineering ---
    
    return df

Hàm `build_windows_multi_ticker()` thực hiện việc chia dữ liệu thành các *windows* thời gian cho mỗi cổ phiếu (ticker), chuẩn bị dữ liệu cho mô hình học máy. Cụ thể, nó tạo các chuỗi con (subsequences) của dữ liệu và nhãn `labels` từ dữ liệu chuỗi thời gian, để có thể huấn luyện các mô hình học sâu (deep learning models) như LSTM.

In [19]:
def build_windows_multi_ticker(df, feature_cols, window=30, step=1):
    all_X, all_y, all_tids, all_times, all_open, all_close, all_prev_close = [], [], [], [], [], [], []
    ticker_to_id = {t: i for i, t in enumerate(sorted(df['ticker'].unique()))}
    for ticker, g in df.groupby('ticker'):
        g = g.sort_values('timestamp').reset_index(drop=True)
        data = g[feature_cols].values
        targets = g['target'].values
        opens = g['open'].values
        closes = g['close'].values
        n = len(g)
        if n <= window:
            continue
        Xs, ys, times, opens_s, closes_s, prev_closes = [], [], [], [], [], []
        for start in range(0, n-window, step):
            end = start + window
            Xs.append(data[start:end])
            ys.append(targets[end])
            times.append(g.loc[end, 'timestamp'])
            opens_s.append(opens[end])
            closes_s.append(closes[end])
            prev_closes.append(closes[end-1])
        if len(ys) == 0:
            continue
        all_X.append(np.array(Xs))
        all_y.append(np.array(ys))
        all_tids.append(np.full(len(ys), ticker_to_id[ticker], dtype=int))
        all_times.append(np.array(times))
        all_open.append(np.array(opens_s))
        all_close.append(np.array(closes_s))
        all_prev_close.append(np.array(prev_closes))
    if len(all_X) == 0:
        return None
    X_all = np.vstack(all_X)
    y_all = np.concatenate(all_y)
    tid_all = np.concatenate(all_tids)
    times_all = np.concatenate(all_times)
    opens_all = np.concatenate(all_open)
    closes_all = np.concatenate(all_close)
    prev_all = np.concatenate(all_prev_close)
    # sort by times to preserve chronology across tickers
    order = np.argsort(times_all)
    return X_all[order], y_all[order], tid_all[order], times_all[order], opens_all[order], closes_all[order], prev_all[order], ticker_to_id


Hàm `build_model()` định nghĩa một mô hình học sâu (deep learning model) sử dụng các lớp **Conv1D**, **LSTM**, **Multi-Head Attention**, và **Embedding** để xử lý chuỗi thời gian với thông tin bổ sung từ cổ phiếu (tickers). Mô hình này phù hợp cho các bài toán phân loại nhị phân, chẳng hạn như dự đoán xu hướng của cổ phiếu (tăng/giảm).

In [20]:
def build_model(window, n_features, n_tickers, emb_dim=8,
                lstm_units=128, conv_filters=32, att_heads=4, att_key_dim=32, dropout=0.5):
    seq_in = Input(shape=(window, n_features), name='seq')
    tick_in = Input(shape=(), dtype='int32', name='ticker')
    
    emb = Embedding(input_dim=n_tickers, output_dim=emb_dim, name='emb')(tick_in)
    emb = Flatten()(emb)
    
    x = Conv1D(filters=conv_filters, kernel_size=3, padding='same', activation='relu', kernel_regularizer=l2(0.001))(seq_in)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(dropout)(x)
    
    x = LSTM(lstm_units, return_sequences=True, kernel_regularizer=l2(0.001))(x)
    x = LayerNormalization()(x)
    
    mha = MultiHeadAttention(num_heads=att_heads, key_dim=att_key_dim)(x, x)
    x = LayerNormalization()(x + mha)
    
    gap = GlobalAveragePooling1D()(x)
    
    merged = Concatenate()([gap, emb])
    d = Dense(64, activation='relu', kernel_regularizer=l2(0.001))(merged)
    d = Dropout(dropout)(d)
    
    # THAY ĐỔI LẠI: 1 unit và activation 'sigmoid' cho phân loại nhị phân
    out = Dense(1, activation='sigmoid')(d)
    
    model = Model(inputs=[seq_in, tick_in], outputs=out)
    return model

<center>
    <h2>LUỒNG CHẠY</h2>
</center>

<center>
    <i>Quy trình đầy đủ: tải dữ liệu, tính toán các chỉ số kỹ thuật, huấn luyện mô hình, cho đến việc đánh giá và đưa ra các tín hiệu mua bán tiềm năng dựa trên các dự đoán của mô hình. Mục tiêu của quy trình này là phát hiện các tín hiệu tiềm năng trong tập dữ liệu thử nghiệm và tạo các báo cáo liên quan đến mô hình.</i>
</center>

In [21]:
def main():
    # Force GPU context
    with tf.device('/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'):
        print("Starting pipeline with device:", tf.config.list_physical_devices('GPU')[0] if tf.config.list_physical_devices('GPU') else 'CPU')
        
        print("Loading data...", CSV_PATH)
        df = load_data(CSV_PATH)
        check_memory()
        
        # Reduce data size if needed
        unique_tickers = df['ticker'].nunique()
        if unique_tickers > 20:  # Limit to 20 tickers for demo
            top_tickers = df['ticker'].value_counts().head(20).index
            df = df[df['ticker'].isin(top_tickers)].reset_index(drop=True)
            print(f"Limited to top {len(top_tickers)} tickers")
        
        # compute technical indicators used later in feature selection
        print("Computing indicators...")
        df = compute_indicators(df)
        check_memory()
        
        print("Rows,cols:", df.shape)
        if START_DATE is not None or END_DATE is not None:
            if START_DATE is not None:
                df = df[df['timestamp'] >= pd.to_datetime(START_DATE)]
            if END_DATE is not None:
                df = df[df['timestamp'] <= pd.to_datetime(END_DATE)]
            df = df.reset_index(drop=True)
            print("Filtered date range ->", df['timestamp'].min(), df['timestamp'].max())

        # drop cols with many missing
        df = drop_high_missing(df, MAX_MISSING_RATIO)

        # numeric cols for interpolation/selection
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        # ensure 'target' etc not in numericcols yet

        # interpolate per ticker
        df = interp_per_ticker(df, numeric_cols)

        # build target using Triple Barrier Method
        # Ensure 'atr_14' is computed in compute_indicators
        df['target'] = apply_triple_barrier(df, 
                                            tp_mult=TAKE_PROFIT_MULT, 
                                            sl_mult=STOP_LOSS_MULT, 
                                            max_period=MAX_HOLD_PERIOD)
        # Drop rows where a label could not be assigned
        df = df.dropna(subset=['target']).copy()
        df['target'] = df['target'].astype(int)

        # ===============================================
        # ======= THÊM DÒNG CODE MỚI VÀO ĐÂY =======
        print(f"Original data size before filtering timeouts: {len(df)}")
        df = df[df['target'] != 2].copy()
        print(f"Data size after filtering timeouts (target=2): {len(df)}")
        # ===============================================

        # time split mask for train-only selection
        train_mask = time_split_train_mask(df, train_frac=0.70)

        # numeric columns used in RFE: exclude target, ret, rf_daily
        num_cols = [c for c in df.select_dtypes(include=[np.number]).columns if c not in ['target','ret','rf_daily']]
        print("Numeric cols available for selection:", len(num_cols))

        # train-only RFE
        selected_rfe, rfe_obj = train_only_rfe(df, num_cols, train_mask, n_select=N_RFE_FEATURES, n_estimators=RFE_ESTIMATORS)
        print("RFE selected:", selected_rfe)

        # # KMeans on selected features (train rows only)
        # X_train_sel = df.loc[train_mask, selected_rfe]
        # labels_cluster, scaler_k, km_obj = kmeans_cluster_features(X_train_sel, n_clusters=N_CLUSTERS)

        # # SOM per cluster selection to get ~FINAL_FEATURE_COUNT features
        # selected_final = som_select_per_cluster(X_train_sel, labels_cluster, selected_rfe, target_count=FINAL_FEATURE_COUNT)
        # print("SOM selected (pre-corr):", selected_final)
        
        selected_final = selected_rfe 
        # correlation filter
        final_feats = correlation_filter(df.loc[train_mask], selected_final, threshold=0.8)
        if len(final_feats) > FINAL_FEATURE_COUNT:
            final_feats = final_feats[:FINAL_FEATURE_COUNT]
        print("Final features after corr filtering:", final_feats)

        # ensure we have features
        if len(final_feats) == 0:
            raise RuntimeError("No features selected - check data and parameters")
        n_features = len(final_feats)
                # Chuẩn hóa dữ liệu theo từng Ticker (Per-Ticker Scaling)
        # Chúng ta sẽ fit scaler trên dữ liệu training của mỗi ticker và transform cho toàn bộ dữ liệu của ticker đó.
        print("Performing per-ticker scaling...")
        df_scaled = df.copy() # Tạo bản sao để chứa dữ liệu đã chuẩn hóa
        for ticker, group in df_scaled.groupby('ticker'):
            # Tạo một scaler riêng cho mỗi ticker
            scaler = StandardScaler()
            
            # Xác định dữ liệu training cho ticker này
            train_data_ticker = group.loc[train_mask[df['ticker'] == ticker], final_feats]
            
            # Fit scaler CHỈ trên dữ liệu training của ticker
            if not train_data_ticker.empty:
                scaler.fit(train_data_ticker)
                
                # Áp dụng scaler đã fit để transform toàn bộ dữ liệu của ticker đó
                group_scaled_values = scaler.transform(group[final_feats])
                
                # Gán lại giá trị đã chuẩn hóa vào DataFrame
                df_scaled.loc[group.index, final_feats] = group_scaled_values

        print("Per-ticker scaling complete.")
        # build multi-ticker windows (also returns ticker mapping)
        res = build_windows_multi_ticker(df_scaled, final_feats, window=WINDOW, step=STEP)
        if res is None:
            raise RuntimeError('No windows created (data too short for given window)')
        X_all, y_all, tids_all, times_all, opens_all, closes_all, prev_all, ticker_to_id = res
        n_total = X_all.shape[0]
        print(f"Built {n_total} samples across {len(ticker_to_id)} tickers")

        # split by chronological order
        train_end = int(n_total * 0.70)
        val_end = int(n_total * 0.85)
        X_train, y_train, t_train = X_all[:train_end], y_all[:train_end], tids_all[:train_end]
        X_val, y_val, t_val = X_all[train_end:val_end], y_all[train_end:val_end], tids_all[train_end:val_end]
        X_test, y_test, t_test = X_all[val_end:], y_all[val_end:], tids_all[val_end:]
        times_test = times_all[val_end:] 
        opens_test = opens_all[val_end:]; closes_test = closes_all[val_end:]; prev_test = prev_all[val_end:]
        print("Split shapes", X_train.shape, X_val.shape, X_test.shape)


        # build model
        n_tickers = len(ticker_to_id)
        model = build_model(WINDOW, n_features, n_tickers, emb_dim=EMB_DIM)
        model.compile(optimizer=AdamW(learning_rate=1e-4, weight_decay=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
        model.summary()

        # Tăng trọng số cho lớp 1 (win) lên, ví dụ gấp 2.5 lần so với lớp 0
        n0 = (y_train == 0).sum()
        n1 = (y_train == 1).sum()
        class_weight_dict = {0: 1.0, 1: 2 * (n0 / n1) if n1 > 0 else 1.0} 
        print('class_weight used in fit (manual adjustment):', class_weight_dict)

        es = EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True)
        ckpt = ModelCheckpoint(os.path.join(OUT_DIR, 'best_model.h5'), monitor='val_loss', mode='min', save_best_only=True)

        # Train with GPU monitoring
        with tf.device('/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'):
            history = model.fit(
                [X_train, t_train], y_train,
                validation_data=([X_val, t_val], y_val),
                epochs=EPOCHS, 
                batch_size=BATCH_SIZE,
                class_weight=class_weight_dict,
                callbacks=[es, ckpt], 
                verbose=2
            )

        # save training plots
        plt.figure(figsize=(10,4))
        plt.plot(history.history.get('loss', []), label='train_loss')
        plt.plot(history.history.get('val_loss', []), label='val_loss')
        plt.legend(); plt.title('Loss'); plt.savefig(os.path.join(OUT_DIR, 'loss.png'))
        plt.close()

        plt.figure(figsize=(10,4))
        plt.plot(history.history.get('accuracy', []), label='train_accuracy')
        plt.plot(history.history.get('val_accuracy', []), label='val_accuracy')
        plt.legend(); plt.title('Accuracy'); plt.savefig(os.path.join(OUT_DIR, 'accuracy.png'))
        plt.close()

        # predict and evaluate
        y_prob = model.predict([X_test, t_test]).ravel() # .ravel() để làm phẳng array
        y_val_prob = model.predict([X_val, t_val]).ravel()
        
        best_threshold = 0.75 # Đặt thẳng ngưỡng
        print(f"\nUsing fixed high threshold: {best_threshold:.2f}")

        # Áp dụng ngưỡng tối ưu lên tập test
        y_pred = (y_prob >= best_threshold).astype(int)
        
        # ==================================================================
        # ======= BẮT ĐẦU CODE MỚI: TÌM VÀ IN CÁC CỔ PHIẾU TIỀM NĂNG =======

        # Tạo một dictionary để map từ ID về lại tên ticker
        id_to_ticker = {v: k for k, v in ticker_to_id.items()}

        # Tạo một DataFrame để tổng hợp kết quả trên tập test
        results_df = pd.DataFrame({
            'timestamp': times_test,
            'ticker_id': t_test,
            'probability': y_prob, # Xác suất thô từ mô hình
            'prediction': y_pred  # Dự đoán cuối cùng (0 hoặc 1)
        })
        results_df['ticker'] = results_df['ticker_id'].map(id_to_ticker)

        # Lọc ra những tín hiệu được dự đoán là "win" (tiềm năng)
        potential_signals = results_df[results_df['prediction'] == 1].copy()
        potential_signals = potential_signals.sort_values(by='timestamp')

        print("\n" + "="*50)
        print("       tín hiệu mua tiềm năng được phát hiện trên tập test")
        print("="*50)

        if potential_signals.empty:
            print("Không tìm thấy tín hiệu nào.")
        else:
            # In ra kết quả
            print(potential_signals[['timestamp', 'ticker', 'probability']].to_string(index=False))

        print("="*50 + "\n")

        # ======= KẾT THÚC CODE MỚI =======
        # ==================================================================
        
        print("\nClassification Report on Test Set using Optimal Threshold:")

        # HIỂN THỊ LẠI BÁO CÁO CHO 2 LỚP
        print(classification_report(y_test, y_pred, target_names=['loss', 'win']))
        cm = confusion_matrix(y_test, y_pred)
        print('Confusion matrix:\n', cm)
        sns.heatmap(cm, annot=True, fmt='d', xticklabels=['loss', 'win'], yticklabels=['loss', 'win']); plt.title('Confusion Matrix'); plt.savefig(os.path.join(OUT_DIR, 'confusion_matrix.png')); plt.close()

        # per-ticker AUC is not directly applicable for multi-class, skipping
        
        # simple backtests on test set using opens/closes stored earlier
        # Only consider 'win' predictions (label 1) for long positions
        profits_tf = np.where(y_pred == 1, closes_test - prev_test, 0.0)
        profits_ls = np.where(y_pred == 1, closes_test - opens_test, 0.0) # Simplified for long-only on win signal
        print('Trend-following cum profit (long on "win" signal):', np.nansum(profits_tf))
        print('Long-short cum profit (long on "win" signal):', np.nansum(profits_ls))

        model.save(os.path.join(OUT_DIR, 'model_final.h5'))
        print('Done. Outputs saved to', OUT_DIR)

In [22]:
if __name__ == '__main__':
    main()

Starting pipeline with device: CPU
Loading data... DSTC_3y.csv
Loading data in chunks...
Memory usage: 80.1% (12.6/15.7 GB)
Memory usage: 80.1% (12.6/15.7 GB)
Memory usage: 80.2% (12.6/15.7 GB)
Data loaded: (22410, 7)
Memory usage: 80.4% (12.6/15.7 GB)
Limited to top 20 tickers
Computing indicators...
Generating advanced features...
Advanced features generated.
Generating interaction, confirmation, and RoC features...
Super advanced features generated.
Memory usage: 64.7% (10.2/15.7 GB)
Rows,cols: (14940, 97)
Dropping 3 columns with >10.0% missing
Applying Triple-Barrier with TP=2*ATR, SL=1*ATR, Max Hold=7 days...
Original data size before filtering timeouts: 14920
Data size after filtering timeouts (target=2): 10830
Time-split train_end date: 2024-10-04T00:00:00.000000000 -> train rows: 7532 / 10830
Numeric cols available for selection: 91
Fitting RFE on train rows... (this may take time)

Feature Ranking by RFE:
Rank 1: adx_10
Rank 1: adx_14
Rank 1: adx_20
Rank 1: adx_neg_10
Rank 1: 

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ seq (InputLayer)              │ (None, 40, 18)            │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d (Conv1D)               │ (None, 40, 32)            │           1,760 │ seq[0][0]                  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling1d (MaxPooling1D)  │ (None, 20, 32)            │               0 │ conv1d[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, 20, 32)            │               0 │ max_pooling1d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm (LSTM)                   │ (None, 20, 128)           │          82,432 │ dropout[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization           │ (None, 20, 128)           │             256 │ lstm[0][0]                 │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ multi_head_attention          │ (None, 20, 128)           │          66,048 │ layer_normalization[0][0], │
│ (MultiHeadAttention)          │                           │                 │ layer_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add (Add)                     │ (None, 20, 128)           │               0 │ layer_normalization[0][0], │
│                               │                           │                 │ multi_head_attention[0][0] │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ ticker (InputLayer)           │ (None)                    │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ layer_normalization_1         │ (None, 20, 128)           │             256 │ add[0][0]                  │
│ (LayerNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ emb (Embedding)               │ (None, 16)                │             320 │ ticker[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_average_pooling1d      │ (None, 128)               │               0 │ layer_normalization_1[0][… │
│ (GlobalAveragePooling1D)      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten (Flatten)             │ (None, 16)                │               0 │ emb[0][0]                  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 144)               │               0 │ global_average_pooling1d[… │
│                               │                           │                 │ flatten[0][0]              │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 160,417 (626.63 KB)

 Trainable params: 160,417 (626.63 KB)

...

110/110 - 11s - 102ms/step - accuracy: 0.4192 - loss: 1.3594 - val_accuracy: 0.4315 - val_loss: 0.9521
Epoch 4/100
110/110 - 12s - 113ms/step - accuracy: 0.4054 - loss: 1.3478 - val_accuracy: 0.4415 - val_loss: 0.9668
Epoch 5/100


110/110 - 14s - 124ms/step - accuracy: 0.4260 - loss: 1.3406 - val_accuracy: 0.4335 - val_loss: 0.9498
Epoch 6/100
110/110 - 12s - 106ms/step - accuracy: 0.4256 - loss: 1.3344 - val_accuracy: 0.4388 - val_loss: 0.9673
Epoch 7/100
110/110 - 13s - 114ms/step - accuracy: 0.4394 - loss: 1.3236 - val_accuracy: 0.4362 - val_loss: 0.9500
Epoch 8/100
110/110 - 12s - 112ms/step - accuracy: 0.4346 - loss: 1.3133 - val_accuracy: 0.4156 - val_loss: 1.0049
Epoch 9/100


110/110 - 13s - 116ms/step - accuracy: 0.4247 - loss: 1.3173 - val_accuracy: 0.4402 - val_loss: 0.9348
Epoch 10/100
110/110 - 11s - 101ms/step - accuracy: 0.4378 - loss: 1.3082 - val_accuracy: 0.4621 - val_loss: 0.9403
Epoch 11/100
110/110 - 12s - 105ms/step - accuracy: 0.4454 - loss: 1.3025 - val_accuracy: 0.4435 - val_loss: 0.9734
Epoch 12/100
110/110 - 11s - 104ms/step - accuracy: 0.4381 - loss: 1.3002 - val_accuracy: 0.4488 - val_loss: 0.9450
Epoch 13/100
110/110 - 11s - 98ms/step - accuracy: 0.4602 - loss: 1.2868 - val_accuracy: 0.4767 - val_loss: 0.9420
Epoch 14/100


110/110 - 12s - 109ms/step - accuracy: 0.4562 - loss: 1.2855 - val_accuracy: 0.4880 - val_loss: 0.9244
Epoch 15/100
110/110 - 12s - 105ms/step - accuracy: 0.4529 - loss: 1.2831 - val_accuracy: 0.4515 - val_loss: 0.9927
Epoch 16/100
110/110 - 11s - 101ms/step - accuracy: 0.4619 - loss: 1.2773 - val_accuracy: 0.4721 - val_loss: 0.9264
Epoch 17/100
110/110 - 11s - 101ms/step - accuracy: 0.4669 - loss: 1.2725 - val_accuracy: 0.4754 - val_loss: 0.9359
Epoch 18/100
110/110 - 11s - 96ms/step - accuracy: 0.4653 - loss: 1.2630 - val_accuracy: 0.4820 - val_loss: 0.9274
Epoch 19/100
110/110 - 11s - 102ms/step - accuracy: 0.4713 - loss: 1.2552 - val_accuracy: 0.4707 - val_loss: 0.9863
48/48 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step

Using fixed high threshold: 0.75

       tín hiệu mua tiềm năng được phát hiện trên tập test
 timestamp ticker  probability
2025-04-01    GAS     0.846616
2025-04-02    FPT     0.762145
2025-04-02    GAS     0.856641
2025-04-03    SAB   

Trend-following cum profit (long on "win" signal): 3412.782800000001
Long-short cum profit (long on "win" signal): 27079.372559999916
Done. Outputs saved to outputs
